<a href="https://colab.research.google.com/github/inayabar/tp2-ia-g16/blob/main/TP2_IA_G16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deap

     |████████████████████████████████| 163kB 5.0MB/s 


In [ ]:
import random
import matplotlib.pyplot as plt
import functools
import numpy as np
import pprint

from deap import base
from deap import creator
from deap import tools

print("Librerías importadas")

Librerías importadas


In [ ]:
# Modelado del INDIVIDUO: Un individuo estará representado por un array de cuatro posiciones, donde 
# cada una de esas posiciones representa a un chico en particular (el índice que representa cada 
# chico está fijado). Así, un individuo tendrá la siguiente forma:
#       [Juan, Daniel, Nicolás, Ricardo]
# A su vez, cada chico (elemento de este array) estará modelado por otro array, esta vez, de cinco elementos
# posiciones, donde cada posición representará uno de los cinco rasgos o características de cada chico. 
# Así, por ejemplo: 
#       [['00', '11', '01', '00', '11'], Daniel, Nicolás, Ricardo]
# Significa que Juan:
#   - Tiene una remera de color rojo
#   - Está ubicado en la primera posición 
#   - Su género de películas favoritas es comedia
#   - Su comida favorita son las papas fritas
#   - Tiene 14 años 

# Tabla de correspondencias: '00' - '01' - '10' - '11'
# - Remera = {negra, azul, verde, roja}
# - Ubicación = {primera posición, segunda posición, tercera posición, cuarta posición}
# - Género (película) = {acción, comedia, horror, thriller}
# - Comida = {papas fritas, galletitas dulces, galletitas saladas, pochoclos}
# - Edad = {11, 12, 13, 14}

In [ ]:
### Constantes ###

# Cantidad de Individuos en la Población
CANT_INDIVIDUOS_POBLACION = 4  
CANT_PROPIEDADES_POR_INDIVIDUO = 5
CANT_REGLAS = 13

[JUAN, DANIEL, NICOLAS, RICARDO] = [0, 1, 2, 3]
[COLOR_REMERA, UBICACION, GENERO_PELICULA, COMIDA, EDAD] = [0, 1, 2, 3, 4]

[ACCION, COMEDIA, HORROR, THRILLER] = ['00', '01', '10', '11']
[UBICACION_1, UBICACION_2, UBICACION_3, UBICACION_4] = ['00', '01', '10', '11']
[NEGRO, AZUL, VERDE, ROJO] = ['00', '01', '10', '11']
[EDAD_11, EDAD_12, EDAD_13, EDAD_14] = ['00', '01', '10', '11']

NOMBRES = [ 'Juan', 'Daniel', 'Nicolas', 'Ricardo' ]
COLORES = [ 'negro', 'azul', 'verde', 'rojo' ]
UBICACIONES = [0,1,2,3]
GENEROS  = [ 'accion', 'comedia', 'horror', 'thriller' ]
COMIDAS  = [ 'papas fritas', 'galletitas dulces', 'galletitas saladas', 'pochoclos']
EDADES    = [ 11,12,13,14 ]

binarios = ['00', '01', '10', '11']

print('Constantes cargadas correctamente.')

Constantes cargadas correctamente.


In [ ]:
### Accessors ###
def ubicacion(individual, name):
  return individual[name][UBICACION]

def color_remera(individual, name): 
  return individual[name][COLOR_REMERA]

def genero_pelicula(individual, name): 
  return individual[name][GENERO_PELICULA]

def comida(individual, name): 
  return individual[name][COMIDA]

def edad(individual, name): 
  return individual[name][EDAD]

print('Accessors cargados correctamente')

Accessors cargados correctamente


In [ ]:
### Helpers ###
def estaEnUnExtremo(ubicacion):
  return ubicacion == UBICACION_1 or ubicacion == UBICACION_4

def leGustaLaComida(tipoComida, chico):
  return chico[COMIDA] == binarios[COMIDAS.index(tipoComida)]

def tieneRemera(colorRemera, chico):
  return chico[COLOR_REMERA] == binarios[COLORES.index(colorRemera)]

def miraElGenero(genero, chico):
  return chico[GENERO_PELICULA] == binarios[GENEROS.index(genero)]

def estaEnLaUbicacion(posicion, chico):
  return chico[UBICACION] == binarios[UBICACIONES.index(posicion)]

def tieneEdad(edad, chico):
  return chico[EDAD] == binarios[EDADES.index(edad)]

def findIndexBy(individual, func):
    for i in range(len(individual)):
        if func(individual[i]):
            return i
    return -1

def flat_map(f, xs):
    ys = []
    for x in xs:
        ys.extend(f(x))
    return ys

def flat_array(array):
  return flat_map(lambda x:x, array)

print('Helpers cargados correctamente.')

def convertirAJson(chico):
  color = COLORES[binarios.index(chico[0])]
  ubicacion = UBICACIONES[binarios.index(chico[1])]
  generoPelicula = GENEROS[binarios.index(chico[2])]
  comida = COMIDAS[binarios.index(chico[3])]
  edad = EDADES[binarios.index(chico[4])]
  return {'ubicacion': ubicacion, 'color': color, 'genero_pelicula':generoPelicula, 'comida':comida, 'edad': edad}

def parsearReglas(individual):
  return {
    'juan_Esta_En_Uno_De_Los_Extremos': juanEstaEnUnoDeLosExtremos(individual),
    'Camisa negra a la izquierda del mas joven':elChicoDeCamisaNegraEstaALaIzquierdaDelMasJoven(individual),
    'A juan le gustan las peliculas de terror':aJuanLeGustanLasPeliculasDeTerror(individual),
    'El chico de 14 esta en la 3 posicion':elChicoDeCatorceAniosEstaEnLaTerceraPosicion(individual),
    'El de camisa roja esta entre el de 13 y el de peliculas de accion':elChicoDeCamisaRojaEstaEntreChicoDeTreceYChicoPeliculasAccion(individual),
    'daniel mira peliculas de suspenso':danielMiraPeliculasDeSuspenso(individual),
    'el que come galletitas dulces esta en un extremo':galletitasDulcesEstaEnUnExtremo(individual),
    'el de remera negra esta a la izquierda del que mira suspenso':remeraNegraALaIzquierdaDeSuspenso(individual),
    'el que come galletitas saladas esta a la derecha del de comedia':chicoQueComeGalletitasSaladasEstaALaDerechaDeComedia(individual),
    'remera Roja Entre Chico Que Come Pochoclos Y Nicolas':remeraRojaEntreChicoQueComePochoclosYNicolas(individual),
    'el que mira suspenso esta en un extremo':suspensoEstaEnUnoDeLosExtremos(individual),
    'nicolas Esta Entre Juan Y Daniel':nicolasEstaEntreJuanYDaniel(individual),
    'primera Posicion tiene remera verda':primeraPosicionCamisetaVerde(individual)               
  }

def parsearMejorIndividuo(individuo40bits):
  # Conversión de array de 40 bits a un array de 4 arrays de 10 bits cada uno
  invididuoAgrupadoPorChico = parsearIndividuo(individuo40bits)
  individual = list(map(parsearChico, invididuoAgrupadoPorChico))
  chicos = list(map(convertirAJson, individual))
  return {
      'Juan': chicos[0],
      'Daniel': chicos[1],
      'Nicolas': chicos[2],
      'Ricardo': chicos[3],
      'reglas': parsearReglas(individual)
  }

Helpers cargados correctamente.


In [ ]:
pp = pprint.PrettyPrinter(indent=4)
ejemplo =  [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0]
pp.pprint(parsearMejorIndividuo(ejemplo))

NameError: ignored

In [ ]:
### Reglas ###

# Regla N°1: Juan está en uno de los extremos.
def juanEstaEnUnoDeLosExtremos(individual):
  return ubicacion(individual, JUAN) == UBICACION_1 or ubicacion(individual, JUAN) == UBICACION_4

# Regla N°2: El niño que lleva la camisa negra está en algún lugar a la izquierda del niño más joven.
def elChicoDeCamisaNegraEstaALaIzquierdaDelMasJoven(individual): 
  indiceCamisaNegra = findIndexBy(individual, functools.partial(tieneRemera, 'negro'))
  indiceMasJoven = findIndexBy(individual, functools.partial(tieneEdad, 11))
  return indiceCamisaNegra < indiceMasJoven

# Regla N°3: A Juan le gustan las películas de terror
def aJuanLeGustanLasPeliculasDeTerror(individual):
  return genero_pelicula(individual, JUAN) == HORROR

# Regla N°4: El chico de 14 años está en la tercera posición
def elChicoDeCatorceAniosEstaEnLaTerceraPosicion(individual):
  indiceTerceraPosicion = findIndexBy(individual, functools.partial(estaEnLaUbicacion, 2))
  chicoTerceraPosicion = individual[indiceTerceraPosicion]
  return tieneEdad(14, chicoTerceraPosicion)

# Regla N°5: El chico que lleva la camiseta roja está en algún lugar entre el chico 
# de 13 años y el que le gustan las películas de acción, en ese orden.
def elChicoDeCamisaRojaEstaEntreChicoDeTreceYChicoPeliculasAccion(individual):
  indiceCamisaRoja = findIndexBy(individual, functools.partial(tieneRemera, 'rojo'))
  indiceChicoDeTrece = findIndexBy(individual, functools.partial(tieneEdad, 13))
  indicePeliculasAccion = findIndexBy(individual, functools.partial(miraElGenero, 'accion'))
  return indiceChicoDeTrece < indiceCamisaRoja < indicePeliculasAccion

# Regla N°6: A Daniel le gustan las películas de suspenso.
def danielMiraPeliculasDeSuspenso(individual):
  return genero_pelicula(individual, DANIEL) == binarios[GENEROS.index('thriller')]

# Regla N°7: El chico que se va a comer galletitas dulces está en uno de los extremos.
def galletitasDulcesEstaEnUnExtremo(individual):
  indiceGalletitasDulces = findIndexBy(individual, functools.partial(leGustaLaComida, 'galletitas dulces'))
  ubicacion = individual[indiceGalletitasDulces][UBICACION]
  return estaEnUnExtremo(ubicacion)

# Regla N°8: El chico que lleva la camiseta negra está exactamente a la izquierda del que le gustan las películas de suspenso.
def remeraNegraALaIzquierdaDeSuspenso(individual):
  indiceRemeraNegra = findIndexBy(individual, functools.partial(tieneRemera,'negro'))

  # valido para evitar out of bounds
  if indiceRemeraNegra == (CANT_INDIVIDUOS_POBLACION - 1):
    return False
  return individual[indiceRemeraNegra + 1][GENERO_PELICULA] == binarios[GENEROS.index('thriller')]
  
# Regla N°9: El chico que se va a comer galletitas saladas está exactamente a la derecha del chico al que le gustan las películas de comedia.
def chicoQueComeGalletitasSaladasEstaALaDerechaDeComedia(individual):
  indiceGalletitasSaladas = findIndexBy(individual, functools.partial(leGustaLaComida, 'galletitas saladas'))

  # valido para evitar out of bounds
  if indiceGalletitasSaladas == 0:
    return False
  return individual[indiceGalletitasSaladas - 1][GENERO_PELICULA] == binarios[GENEROS.index('comedia')]

# Regla N°10: El chico que lleva la camiseta roja está en algún lugar entre el chico que va a comer pochoclos y Nicolás, en ese orden.
def remeraRojaEntreChicoQueComePochoclosYNicolas(individual):
  indiceRemeraRoja = findIndexBy(individual, functools.partial(tieneRemera, 'rojo'))
  if indiceRemeraRoja == (CANT_INDIVIDUOS_POBLACION - 1) or indiceRemeraRoja == 0: # Esta en un extremo
    return False
  personaAnterior = individual[indiceRemeraRoja-1]
  personaPosterior = individual[indiceRemeraRoja+1]
  return (personaAnterior == leGustaLaComida('pochoclos', personaAnterior)) and (personaPosterior == NICOLAS)

# Regla N°11: En uno de los extremos está el chico al que le gustan las películas de suspenso.
def suspensoEstaEnUnoDeLosExtremos(individual):
  indiceSuspenso = findIndexBy(individual, functools.partial(miraElGenero, 'thriller'))
  ubicacion = individual[indiceSuspenso][UBICACION]
  return estaEnUnExtremo(ubicacion)

# Regla N°12: Nicolás está en algún lugar entre Juan y Daniel, en ese orden.
def nicolasEstaEntreJuanYDaniel(individual):
  ubicacionNicolas = ubicacion(individual, NICOLAS)
  ubicacionJuan = ubicacion(individual, JUAN)
  ubicacionDaniel = ubicacion(individual, DANIEL)
  return ubicacionNicolas > ubicacionJuan and ubicacionNicolas < ubicacionDaniel

# Regla N°13: En la primera posición está el niño con la camiseta verde.
def primeraPosicionCamisetaVerde(individual):
  indiceRemeraVerde = findIndexBy(individual, functools.partial(tieneRemera, 'verde'))
  indicePrimeraPosicion = findIndexBy(individual, functools.partial(estaEnLaUbicacion, 0))
  return indiceRemeraVerde == indicePrimeraPosicion

print("Reglas cargadas correctamente.")

In [ ]:
### Funcion de aptitud (fitness) ###
import itertools

def parsearIndividuo(individuo):
  for i in range(0, len(individuo), 10):
    yield individuo[i:i + 10]

def parsearChico(chico):
  result = []
  lastIndex = 0
  for _ in itertools.repeat(None, CANT_PROPIEDADES_POR_INDIVIDUO):
    propiedad = ''
    propiedad += str(chico[lastIndex])
    propiedad += str(chico[lastIndex + 1])
    lastIndex = lastIndex + 2
    result.append(propiedad)
  return(result)

def funcionAptitud(individuo40bits):
  # Conversión de array de 40 bits a un array de 4 arrays de 10 bits cada uno
  invididuoAgrupadoPorChico = parsearIndividuo(individuo40bits)
  individual = list(map(parsearChico, invididuoAgrupadoPorChico))

  reglasAplicadas = [
    juanEstaEnUnoDeLosExtremos(individual),
    elChicoDeCamisaNegraEstaALaIzquierdaDelMasJoven(individual),
    aJuanLeGustanLasPeliculasDeTerror(individual),
    elChicoDeCatorceAniosEstaEnLaTerceraPosicion(individual),
    elChicoDeCamisaRojaEstaEntreChicoDeTreceYChicoPeliculasAccion(individual),
    danielMiraPeliculasDeSuspenso(individual),
    galletitasDulcesEstaEnUnExtremo(individual),
    remeraNegraALaIzquierdaDeSuspenso(individual),
    chicoQueComeGalletitasSaladasEstaALaDerechaDeComedia(individual),
    remeraRojaEntreChicoQueComePochoclosYNicolas(individual),
    suspensoEstaEnUnoDeLosExtremos(individual),
    nicolasEstaEntreJuanYDaniel(individual),
    primeraPosicionCamisetaVerde(individual)               
  ]
  
  suma = sum(list(map(lambda validRule: 1 if validRule else 0, reglasAplicadas)))
  return suma,  

print("Funcion de fitness cargada correctamente.")    

In [ ]:
### Creación de instancia toolbox ###
toolbox = base.Toolbox()

In [ ]:
### Definición de operadores genéticos (POR DEFAULT) ### 

# Métodos para los peradores genéticos 
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("crossover", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.5)

In [ ]:
### Definición de operadores genéticos (SLIDERS) ### 

# Definicion del tipo de seleccion y cruzamiento 
tipo_seleccion = 'Ranking' #@param ["Azar", "Torneo", "Ranking", "Ruleta", "Control s/Número Esperado"]
tipo_cruzamiento = 'Simple-1Punto' #@param ["Simple-1Punto", "Simple-2Puntos", "Azar"]
tipo_mutacion = 'MutFliptBit' #@param ["MutFliptBit", "MutGaussian"]

# Establecer tipo de seleccion
if tipo_seleccion == 'Torneo':
  toolbox.register("select", tools.selTournament, tournsize=3)
elif tipo_seleccion == 'Ranking':
  toolbox.register("select", tools.selBest)
elif tipo_seleccion == 'Ruleta':
  toolbox.register("select", tools.selRoulette)
elif tipo_seleccion == 'Control s/Número Esperado':
  toolbox.register("select", tools.selStochasticUniversalSampling)
else:
  toolbox.register("select", tools.selRandom)

# Establecer tipo de cruza
if tipo_cruzamiento == 'Simple-1Punto':
  toolbox.register("crossover", tools.cxOnePoint)
elif tipo_cruzamiento == 'Simple-2Puntos':
  toolbox.register("crossover",  tools.cxTwoPoint)  
else:
  toolbox.register("crossover", tools.cxUniform, indpb=0.5)

# Establecer tipo de mutación
if tipo_mutacion == 'MutFliptBit':
  toolbox.register("mutate", tools.mutFlipBit, indpb=0.5)
if tipo_mutacion == 'MutGaussian':
  toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.2, indpb=0.2)

# Probabilidad de cruza
P_CROSSOVER = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}

# Probabilidad del Mutación 
P_MUTATION = 0.4 #@param {type:"slider", min:0, max:1, step:0.1}

# Establecer tipo de mutación
P_MUTATION_BIT = 0.2 #@param {type:"slider", min:0, max:1, step:0.1}
toolbox.register("mutate", tools.mutFlipBit, indpb=P_MUTATION_BIT)

# Variables del algoritmo genético
# number of individuals in population
POPULATION_SIZE = 50 #@param {type:"slider", min:10, max:100, step:10}
# max number of generation for stopping condition
MAX_GENERATIONS = 500 #@param {type:"slider", min:50, max:1000, step:50}

In [ ]:
# Definiciones del algoritmo genético

# Descomentar para tener siempre los mismos resultados (para pruebas)
# RANDOM_SEED = 42
# random.seed(RANDOM_SEED)

# Registramos la función auxiliar zeroOrOne al toolbox
toolbox.register("zeroOrOne", random.randint, 0, 1)

# Creamos la clase FitnessMax 
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Creamos la clase Individual que representa a un individuo de nuestra población
creator.create("Individual", list, fitness=creator.FitnessMax)

CANTIDAD_CHICOS = 4
CANTIDAD_PROPIEDADES = 5

# Registramos la función para crear individuos en el toolbox
toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.zeroOrOne, CANTIDAD_CHICOS * CANTIDAD_PROPIEDADES * 2)

# Registramos la función para crear la población en el toolbox
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

# Registramos la función de fitness en el toolbox
toolbox.register("evaluate", funcionAptitud)

In [ ]:
# BORRADOR Cruza
population = toolbox.populationCreator(2)
childAOriginal = population[0]
childBOriginal = population[1]
print('childA original: {}'.format(childAOriginal))
print('childB original: {}'.format(childBOriginal))
for childA, childB in zip (population[::2], population[1::2]): 
  #if random.random() < P_CROSSOVER: # Se decide si se hace la cruza o no. Ejemplo: random.random() => 0.5 < P_CROSSOVER = 0.9 => Se hace la cruza. random.random() retorna un valor decimal entre 0 y 1
  result = toolbox.crossover([1,0,0,0], [0,1,1,1]) # La función crossover modifica los individuos "in place", tiene efecto de lado
  print(result)

In [ ]:
# BORRADOR Cruza
population = toolbox.populationCreator(2)
childAOriginal = population[0]
childBOriginal = population[1]
print('childA original: {}'.format(childAOriginal))
print('childB original: {}'.format(childBOriginal))
for childA, childB in zip (population[::2], population[1::2]): 
  #if random.random() < P_CROSSOVER: # Se decide si se hace la cruza o no. Ejemplo: random.random() => 0.5 < P_CROSSOVER = 0.9 => Se hace la cruza. random.random() retorna un valor decimal entre 0 y 1
  toolbox.crossover(childA, childB) # La función crossover modifica los individuos "in place", tiene efecto de lado
  del childA.fitness.values # Debemos eliminar los valores de fitness de los hijos modificados ya que los mismos han sido alterados y no siguen siendo válidos
  del childB.fitness.values
  #print('childA: {}'.format(childA2))
  print('childA cruzado: {}'.format(population[0]))
  print(childAOriginal == population[0])
  #print('childB: {}'.format(childB2))
  print(childBOriginal == population[1])
  print('childB cruzado: {}'.format((population[1])))

In [ ]:
############ ESTRUCTURA DEL ALGORITMO GENÉTICO ############
# 1. Creación de la población inicial
#population = toolbox.populationCreator(POPULATION_SIZE)
population = toolbox.populationCreator(3)
generationCounter = 0

In [ ]:
# 2. Aplicación de la función de fitness (a cada individuo de la población)
fitnessValues = list(map(toolbox.evaluate, population))

In [ ]:
# 3. Asociamos los individuos con su respectivo valor de fitness y armamos una lista de tuplas (individuo, valorFitness)
# Recordar que un objeto Individual tiene un atributo de instancia llamado fitness del tipo creator.FitnessMax
for individual, fitnessValue in zip(population, fitnessValues):
  individual.fitness.values = fitnessValue

In [ ]:
# 4. Obtenemos el primer valor de cada tupla de fitness (el único que usamos) para fines estadísticos
fitnessValues = [individual.fitness.values[0] for individual in population]

In [ ]:
# 5. Las estadísticas que vamos a considerar son el mayor valor de fitness y el promedio del valor de fitness de todos los individuos.
# Ambas métricas se calcularán por cada generación, así que finalmente tendremos una lista de valores por cada una.
maxFitnessValues = []
meanFitnessValues = []
bestIndividualsEachGeneration = [] 

In [ ]:
# 6. Comenzamos a definir el bucle principal del algoritmo genético. 
# Lo primero que definimos son las condiciones de paro. En este caso tenemos dos: cuando se alcanza el número máximo de generaciones (MAX_GENERATIONS) o cuando
# se obtiene un individuo que maximiza la función de fitness (funcionAptitud), es decir, alcanza un valor igual al número de reglas (CANTIDAD_REGLAS) ya que las cumple todas.
CANTIDAD_REGLAS = 13
while max(fitnessValues) < CANTIDAD_REGLAS and generationCounter < MAX_GENERATIONS:
  # 7. Incrementamos en uno el contador de generaciones
  generationCounter = generationCounter + 1
  # 8. SELECCION: Aplicamos el operador de selección a la población actual // offspring = descendencia, hijos
  offspring = toolbox.select(population, len(population))
  
  # 9. Clonamos la población filtrada (almacenada en la variable offspring) para poder aplicar los siguientes operadores genéticos sin alterar la población original
  offspring = list(map(toolbox.clone, offspring))
  # Nota: Si bien llamamos a esta lista offspring, en realidad todavía son clones de la generación actual. Para tener realmente descedencia debemos aplicar el operador de cruza
  # 10. CRUZA: Aplicamos el operador de cruza (si se cumple la probabilidad)
  for individualA, individualB in zip (offspring[::2], offspring[1::2]): 
    if random.random() < P_CROSSOVER: # Se decide si se hace la cruza o no. Ejemplo: random.random() => 0.5 < P_CROSSOVER = 0.9 => Se hace la cruza. random.random() retorna un valor decimal entre 0 y 1
      toolbox.crossover(individualA, individualB) # La función crossover modifica los individuos "in place", tiene efecto de lado
      # Debemos eliminar los valores de fitness de los hijos modificados ya que los mismos han sido alterados y no siguen siendo válidos
      del individualA.fitness.values 
      del individualB.fitness.values
  # 11. MUTACIÓN: Aplicamos el operador de mutación (si se cumple la probabilidad)
  for candidateToMutate in offspring:
    if random.random() < P_MUTATION:
      if random.random() < 0.5:
        toolbox.mutate(candidateToMutate) # La función mutate tiene efecto de lado, muta el individuo "in place". Por ejemplo, el individuo [1,0,1,0,0] podría resultar en el individuo [1,1,1,0,0]
          # Nuevamente, debemos eliminar los valores de fitness del individuo que modificamos (en este caso mutamos) porque no sigue siendo válido: según el ejemplo anterior paso de valer 2 a valer 3, por lo que debe ser recalculada
        del candidateToMutate.fitness.values
  # 12. Los individuos que no se seleccionan para la cruza o la mutación permanecen inalterados, por lo que su valor de fitness se mantiene intacto y no necesita ser 
  # recalculado (hacerlo sería un desperdicio de procesamiento). Los individuos que hayan sido alterados tendrán su valor de fitness eliminado (porque así lo hicimos con del) y el mismo deberá 
  # ser calculado. Para decidir, entonces, sobre que individuos de la población resultante es necesario recalcular la función de fitness vamos a utilizar la propiedad fitness
  # del objeto Individual: si está vacía la calculamos, sino la dejamos como está
  freshIndividuals = [ind for ind in offspring if not ind.fitness.valid]
  freshFitnessValues = list (map(toolbox.evaluate, freshIndividuals))
  for individual, fitnessValue in zip(freshIndividuals, freshFitnessValues):
    individual.fitness.values = fitnessValue
  # 13. Reemplazamos la generación (población) actual con la nueva
  population[:] = offspring
  # 14. Recolectamos los valores de fitness actuales para las estadísticas. Dado que el valor de fitness de cada individuo es sólo el primer elemento de la tupla, lo seleccionamos usando [0]
  fitnessValues = [ind.fitness.values[0] for ind in population]
  # 15. Calculamos el valor de fitness máximo y el promedio de la presente generación
  maxFitness = max(fitnessValues)
  meanFitness = sum(fitnessValues) / len(population)
  maxFitnessValues.append(maxFitness)
  meanFitnessValues.append(meanFitness)
  print("- Generation {}: MAX Fitness = {}, AVG Fitness = {}".format(generationCounter, maxFitness, meanFitness))
  # 16. También imprimimos el mejor individuo de la presente generación (el que tiene mayor valor de fitness, es decir, mayor cantidad de 1s)
  bestIndex = fitnessValues.index(max(fitnessValues))
  print("Best Individual = ", *population[bestIndex], "\n")
  # Agregamos el mejor individuo de la presente generación junto a su valor de fitness a la lista de mejores individuos
  bestIndividualsEachGeneration.append((maxFitness, population[bestIndex]))

# 18. Una vez que se cumple alguna condición de paro y el algoritmo genético mostramos las estadísticas de la corrida
# Mejor individuo de toda la corrida (el mejor de todas las generaciones)
mejorIndividuo = max(bestIndividualsEachGeneration, key = lambda i:i[0])
print('Mejor Individuo de todas las generaciones: {}'.format(mejorIndividuo[1]))
print('Mejor valor de Fitness de todas las generaciones: {}'.format(mejorIndividuo[0]))
print('Mejor Individuo de todas las generaciones (JSON): {}'.format(parsearMejorIndividuo(mejorIndividuo[1])))
# Grafico
plt.plot(maxFitnessValues, color='red')
plt.plot(meanFitnessValues, color='green')
plt.xlabel('Generación')
plt.ylabel('Máximo / Promedio Fitness')
plt.title('Máximo y Promedio Fitness sobre generaciones')
plt.show()
